<a href="https://colab.research.google.com/github/ramrajv/NIFTY_500_Sentiment_Analysis/blob/main/playbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [2]:
pip install nsepython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [19]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from nsepython import *
import datetime
import pytz
#import time

import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Get Company Sector, industry data

In [16]:
nifty_500_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
nifty_200_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty200list.csv'
nifty_100_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty100list.csv'
nifty_50_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
tickers_file = pd.read_csv(nifty_500_ticker_url)
tickers_df = tickers_file[['Symbol', 'Company Name']]
tickers = tickers_df['Symbol']

In [17]:
# Get articles and put them into a df
news_url = 'https://ticker.finology.in/company/'

In [27]:
# list to store article data
data = []
unavailable_tickers = []
companies_len = len(tickers)
length = companies_len
days_limit = datetime.datetime.now() - datetime.timedelta(days=30) #only 30 days old or newer articles
print('Fetching Article data..')
for i in range(length):
    print(i, tickers[i])
    url= '{}/{}'.format(news_url, tickers[i])
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.content, 'lxml')
    news_links = html.select('#newsarticles > a')
    if len(news_links) == 0:
        print('No news found for {}'.format(tickers[i]))
        unavailable_tickers.append(tickers[i])
        continue
    new_articles_counter = 0
    for link in news_links:
        title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        if (date_time_obj <= days_limit):
             continue
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        data.append([tickers[i], title, art_date, art_time])
        new_articles_counter += 1
    if(new_articles_counter==0):
        unavailable_tickers.append(tickers[i])  

df = pd.DataFrame(data, columns=['Ticker', 'Headline', 'Date', 'Time'])

print(unavailable_tickers)

Fetching Article data..
0 360ONE
1 AARTIDRUGS
2 AAVAS
3 ABB
4 ABCAPITAL
5 ABFRL
6 ACC
7 ADANIENT
8 ADANIGREEN
9 ADANIPORTS
10 ADANITRANS
11 AEGISCHEM
12 AETHER
13 AFFLE
14 AIAENG
15 AJANTPHARM
16 ALKEM
17 ALOKINDS
18 AMBUJACEM
19 ANGELONE
20 ANURAS
21 APLLTD
22 APOLLOHOSP
23 ASAHIINDIA
24 ASHOKLEY
25 ASIANPAINT
26 ASTERDM
27 ASTRAL
28 ATGL
29 ATUL
30 AUBANK
31 AUROPHARMA
32 AVANTIFEED
33 AWL
34 AXISBANK
35 BAJAJ-AUTO
36 BAJAJFINSV
37 BAJAJHLDNG
38 BAJFINANCE
39 BALAMINES
40 BALRAMCHIN
41 BANDHANBNK
42 BANKBARODA
43 BANKINDIA
44 BASF
45 BBTC
46 BCG
47 BDL
48 BEL
49 BHARATFORG
50 BHARTIARTL
51 BHEL
52 BIOCON
53 BIRLACORPN
54 BLUESTARCO
55 BORORENEW
56 BOSCHLTD
57 BPCL
58 BRIGADE
59 BSE
60 BSOFT
61 CAMS
62 CANBK
63 CANFINHOME
64 CAPLIPOINT
65 CARBORUNIV
66 CASTROLIND
67 CCL
68 CENTRALBK
69 CENTURYTEX
70 CERA
71 CGCL
72 CGPOWER
73 CHALET
74 CHAMBLFERT
75 CHEMPLASTS
76 CHOLAFIN
77 CIPLA
78 CLEAN
79 COALINDIA
80 COCHINSHIP
81 COFORGE
82 CONCOR
83 COROMANDEL
84 CREDITACC
85 CRISIL
86 CROMPTON

## removing unavailable tickers from original df

In [28]:
tickers = np.setdiff1d(tickers, unavailable_tickers)
tickers_df.drop(tickers_df[tickers_df['Symbol'].isin(unavailable_tickers)].index.values, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Sentiment Analysis

In [29]:
print('Performing Sentiment Analysis')
vader = SentimentIntensityAnalyzer()
scores = df['Headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)
new_df = pd.merge(left=df, right=scores_df, on=df.index.values).drop(['key_0'], axis=1)
final_df = new_df.groupby('Ticker').mean()

Performing Sentiment Analysis


# Get Company,Sector,Industry,mCap Data

In [30]:
tickers[0]

'360ONE'

In [31]:
len(tickers)

374

In [32]:
sector = []
industry = []
mCap = []
companyName = []
new_length = len(tickers)
#new_length = 10
print('Fetching industry data')
for i in range(new_length):
    meta = nse_eq(tickers[i])
    print(i, tickers[i])
    try:
        sector.append(meta['industryInfo']['macro'])
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        sector.append(np.nan)
        industry.append(np.nan)
        mCap.append(np.nan)
        companyName.append(np.nan)
        continue
    #pprint('Sector: {}'.format(meta['industryInfo']['macro']))
    try:
        industry.append(meta['industryInfo']['sector'])
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        industry.append(np.nan)
        mCap.append(np.nan)
        companyName.append(np.nan)
        continue
    #pprint('Industry: {}'.format(meta['industryInfo']['sector']))
    try:
        mCap.append(round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2))
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        mCap.append(np.nan)
        companyName.append(np.nan)
        continue
    try:
        companyName.append(meta['info']['companyName'])
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        companyName.append(np.nan)
    
    #print('market cap is Rs {}'.format(ticker_mcap))
    #print('\n')

final_df['mCap (Billion)'] = mCap
final_df['Company Name'] = companyName

Fetching industry data
0 360ONE
1 AARTIDRUGS
2 AAVAS
3 ABB
4 ABCAPITAL
5 ABFRL
6 ACC
7 ADANIENT
8 ADANIGREEN
9 ADANIPORTS
10 ADANITRANS
11 AEGISCHEM
12 AETHER
13 AFFLE
14 AIAENG
15 AJANTPHARM
16 ALKEM
17 ALOKINDS
18 AMBUJACEM
19 ANGELONE
20 ANURAS
21 APLLTD
22 APOLLOHOSP
23 ASAHIINDIA
24 ASHOKLEY
25 ASIANPAINT
26 ASTERDM
27 ASTRAL
28 ATGL
29 ATUL
30 AUBANK
31 AUROPHARMA
32 AVANTIFEED
33 AWL
34 AXISBANK
35 BAJAJ-AUTO
36 BAJAJFINSV
37 BAJAJHLDNG
38 BAJFINANCE
39 BALAMINES
40 BALRAMCHIN
41 BANDHANBNK
42 BANKBARODA
43 BANKINDIA
44 BASF
45 BBTC
46 BCG
47 BDL
48 BEL
49 BHARATFORG
50 BHARTIARTL
51 BHEL
52 BIOCON
53 BIRLACORPN
54 BLUESTARCO
55 BORORENEW
56 BOSCHLTD
57 BPCL
58 BRIGADE
59 BSE
60 BSOFT
61 CAMS
62 CANBK
63 CANFINHOME
64 CAPLIPOINT
65 CARBORUNIV
66 CASTROLIND
67 CCL
68 CENTRALBK
69 CENTURYTEX
70 CERA
71 CGCL
72 CGPOWER
73 CHALET
74 CHAMBLFERT
75 CHEMPLASTS
76 CHOLAFIN
77 CIPLA
78 CLEAN
79 COALINDIA
80 COCHINSHIP
81 COFORGE
82 CONCOR
83 COROMANDEL
84 CREDITACC
85 CRISIL
86 CROMPTON


In [33]:
final_df = final_df.reset_index()
# final_df = pd.merge(final_df, xc_indices, left_on='Ticker', right_on='Ticker', how='inner')
final_df.columns = ['Symbol', 'Negative', 'Neutral', 'Positive', 'Sentiment Score','MCap (Billion)', 'Company Name']

In [34]:
final_df = final_df.dropna()
final_df = final_df.round(3)

final_df.to_csv('sentiment_data.csv')

In [37]:
# Plotting
print('Generating Plots')
fig = px.treemap(
    final_df, path=[px.Constant('Nifty 500'), 'Symbol'], color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
#values='MCap (Billion)', 
fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)

Generating Plots


In [44]:
sentiscore = final_df[['Company Name','Sentiment Score']]
sentiscore = sentiscore.sort_values('Sentiment Score', ascending = False)

In [45]:
sentiscore.to_csv('SentimentalScore.csv')

In [43]:
# import kaleido
fig.write_html('image.html')

In [36]:
# Get current date, time and timezone to print to the html page
# now = datetime.datetime.now()
# ist_timezone = pytz.timezone('Asia/Kolkata')
# dt_string = now.astimezone(ist_timezone).strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
# Generate HTML File with Updated Time and Treemap
# print('Writing HTML')
# with open('NIFTY_500_live_sentiment.html', 'a') as f:
#     f.truncate(0) # clear file if something is already written on it
#     title = "<h1>NIFTY 500 Stock Sentiment Dashboard</h1>"
#     updated = "<h2>Last updated: " + dt_string + " (Timezone: IST" + ")</h2>"
#     description = "This dashboard is updated at 17:30 IST Every Day with sentiment analysis performed on latest scraped news headlines.<br><br>"
#     f.write(title + updated + description)
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn')) # write the fig created above into the html file